<a href="https://colab.research.google.com/github/clara-lan/Capstone/blob/SVM_YOLO_Object/CapstoneSVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use SVM to detect empty space and pre-process images

# Step1: Import system and dependancies

In [1]:
import os
from os import listdir
from os.path import isfile, join
!pip install roboflow
from roboflow import Roboflow
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer
import tensorflow as tf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 145 kB 7.5 MB/s 
     |████████████████████████████████| 178 kB 55.4 MB/s 
     |████████████████████████████████| 67 kB 4.5 MB/s 
     |████████████████████████████████| 54 kB 2.3 MB/s 
     |████████████████████████████████| 138 kB 44.5 MB/s 
     |████████████████████████████████| 62 kB 999 kB/s 
  Created wheel for roboflow: filename=roboflow-0.2.17-py3-none-any.whl size=31935 sha256=546e54d43a14affc2489cf3c198c5fdb444109e49cf154c612c931f5d84ac7b2
  Stored in directory: /root/.cache/pip/wheels/f2/38/3c/b4ac4d8a9d9b44bdcd51f6148ec810b0f05a404e5fed8df48d
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=8d27c00eeceaf655fc77934380e96ae51137a6283bbe414ec880f7f5959eb966
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built roboflow wget
  Attempting un

In [2]:
import pandas as pd
import cv2
import sys
import math
import time
from sklearn import svm
from sklearn.metrics import accuracy_score
from skimage.feature import hog

## Import dataset from Roboflow

In [3]:
from skimage import feature

In [4]:
import PIL

In [5]:
from sklearn.utils.fixes import sklearn
import tensorflow.python.platform
from tensorflow.python.platform import gfile

In [7]:
"""Read method1: Load roboflow dataset from Roboflow in tfrecord"""
rf = Roboflow(api_key="cKIuGvQRsLbBvFgxNztc")
project = rf.workspace("myworkspace-nfnwm").project("projectdb")


loading Roboflow workspace...
loading Roboflow project...


In [ ]:
dataset = project.version(2).download("tfrecord")

In [ ]:
"""Download dataset in YOLOv7 object"""
!pwd
yolo_dataset = project.version(2).download("yolov7")

In [9]:
"""Read images from tf record"""
def extract_images(path,mode):
  """
    Function: extract_images
    Input: path - valid image path in tf record
           mode - 1 for training, 2 for test
    Output: image dict, {key: {image, label}}, key is index
    Description: This function extract images and labels from tfrecord, maps them to corresponding index
  """
  images = {}
  # Create a description of the features.
  features = {
      "image/encoded": tf.io.FixedLenFeature([], tf.string),
      "image/format": tf.io.FixedLenFeature([], tf.string),
      "image/height": tf.io.FixedLenFeature([], tf.int64),
      "image/object/bbox/xmax": tf.io.VarLenFeature(tf.float32),
      "image/object/bbox/xmin": tf.io.VarLenFeature(tf.float32),
      "image/object/bbox/ymax": tf.io.VarLenFeature(tf.float32),
      "image/object/bbox/ymin": tf.io.VarLenFeature(tf.float32),
      "image/object/class/label": tf.io.VarLenFeature(tf.int64),
      "image/object/class/text": tf.io.VarLenFeature(tf.string),
      "image/width": tf.io.FixedLenFeature([], tf.int64),
  }
  size = 0
  idx = 0
  if mode == 1:
    size = 400
  elif mode == 2:
    size = 100
  train_dataset = tf.data.TFRecordDataset(path)
  for raw_record in train_dataset.take(size):
    sample = tf.io.parse_single_example(raw_record, features)
    image = tf.image.decode_image(sample['image/encoded'], dtype=tf.float32) 
    label = sample['image/object/class/label']
    images[idx] = [image, label]
    idx+=1
  return images
# path =  '/content/Capstone-3/train/empty-shelf-space.tfrecord'
# extract_images(path, 1)

In [10]:
"""Read images from YOLOv7"""

def extract_images_from_yolov7(path,mode):
  """
    Function: extract_images
    Input: path - valid image path in tf record
           mode - 1 for training, 2 for test
    Output: image dict, {key: {image, label}}, key is index, image is the images name, label is the name of label file
  """
  image_labels_dict = {}
  images_path = path+"/images"
  labels_path = path+'/labels'
  images = [f for f in os.listdir(images_path)]
  labels = [f for f in os.listdir(labels_path)]
  images.sort()
  labels.sort()
  for i in range(len(images)):
    image_labels_dict[i] = [images[i], labels[i]]
    # print(images[i])
    # # print("tuple: ",images[i], "====tuple:", labels[i])
    # return

  return image_labels_dict

# extract_images_from_yolov7('/content/ProjectDB-2/test',1)

# Step2: Preprocess data

In [ ]:
"""Convert Tensor to image
  Show Image
"""
def show_image_from_tensor(tensor):
    tensor = tensor*255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor)>3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]
        img_arr = PIL.Image.fromarray(tensor)
        plt.imshow(img_arr, interpolation='nearest')
        plt.show()


In [11]:
# test_dataset = tf.data.TFRecordDataset('/content/Empty-shelf-detection-15/test/Empty-space.tfrecord')
# valid_dataset = tf.data.TFRecordDataset('/content/Empty-shelf-detection-15/valid/Empty-space.tfrecord')
train_labels = '/content/ProjectDB-2/train/empty-shelf-space_label_map.pbtxt'
# valid_labels = '/content/Empty-shelf-detection-15/valid/Empty-space_label_map.pbtxt'

# Step3: Train SVM
---



In [12]:
def datagen(mode):
    """
    Function: datagen 
    
    Input: 
        mode - 1 denotes train data ; 2 denotes test data
    
    Output: Train/Test data and labels depending on mode value
    
    Description: This function computes HOG features for each image in the Dataset/train or Dataset/test folder, assigns label to the descriptor vector of the image and returns the final train/test data and labels matrices used for feeding the SVM in training phase or predicting the label of test data.
    
    """

    X = []
    y = []
    ind = 0
    if mode == 1:
        path = '/content/ProjectDB-2/train/empty-shelf-space.tfrecord'
    elif mode == 2:
        path = '/content/ProjectDB-2/valid/empty-shelf-space.tfrecord'
    # read image
    images = extract_images(path, mode)
    for key in images.keys():
        # compute HOG features
        img, label = images[key][0], images[key][1]
        # print(type(img))
        des, hog_image = hog(img, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(4, 4), block_norm= 'L2',visualize=True)
        # print("des:",des)
        # print(hog_image)
        # print("des: ", des)
        """Read labels"""
        # cv2_imshow(hog_image)
        # append descriptor and label to train/test data, labels
        label = tf.sparse.to_dense(label).numpy().tolist()
        if(len(label) > 0):
          X.append(des)
          y.append(label)
    return X, y
# datagen(2)
# x,y = datagen(1)
# print("y: ", y)


In [92]:
import ast
"""Read labels from .txt file"""
def read_labels(filename):
  labels = [x.split(' ') for x in open(filename).readlines()]
  for label in labels:
    label[-1].rstrip()
  return labels


In [89]:
from google.colab.patches import cv2_imshow
from random import randrange
def datagen_yolov7(mode):
    """
    Function: datagen_yolov7 
    
    Input: 
        mode - 1 denotes train data ; 2 denotes test data
    
    Output: Train/Test data and labels depending on mode value
    
    Description: This function computes HOG features for each image in the Dataset/train or Dataset/test folder, assigns label to the descriptor vector of the image and returns the final train/test data and labels matrices used for feeding the SVM in training phase or predicting the label of test data.
    
    """

    X = []
    y = []
    ind = 0
    if mode == 1:
        path = '/content/ProjectDB-2/train'
    elif mode == 2:
        path = '/content/ProjectDB-2/valid'
    # read image
    image_labels_dict = extract_images_from_yolov7(path, mode)
    for key in image_labels_dict.keys():
        # compute HOG features
        img, label = image_labels_dict[key][0], image_labels_dict[key][1]
        # print(img)
        image = cv2.imread(path+'/images/'+img)
        # cv2_imshow(image)
        # return
        img = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img,(128,128))

        # print(type(img))
        des, hog_image = hog(img, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(4, 4), block_norm= 'L2',visualize=True)
        # print("des:",des)
        # print(hog_image)
        # print("des: ", des)
        """Read labels"""
        # cv2_imshow(hog_image)
        # append descriptor and label to train/test data, labels
        X.append(des)
        # print(des)
        label = read_labels(path+'/labels/'+label)
        #store only one label
        y.append(label[randrange(len(label))])
    return X, y

# datagen_yolov7(2)

In [ ]:
# def train_svm():
#     # list of training and test files
#     # call 'datagen' function to get training and testing data & labels
#     mlb = MultiLabelBinarizer()
#     Xtrain, ytrain = datagen(1)
#     Xtest, ytest = datagen(2)

#     # print("ytrain: ", ytrain)
#     # convert matrices to numpy array for fast computation
#     Xtrain_arr = np.array(Xtrain)
#     # convert labels to 1d array to enabel sklearn
#     ytrain = [item[0] for item in ytrain]
#     # print("x: ", len(Xtrain))
#     # print("ytrain: ", ytrain)
#     Xtest_arr = np.array(Xtest)
#     ytest = [item[0] for item in ytest]

#     # training phase: SVM , fit model to training data ------------------------------
#     clf = svm.SVC(kernel = 'linear')
#     clf.fit(Xtrain_arr, ytrain)
#     # predict labels for test data
#     ypred = clf.predict(Xtest_arr)
    
#     # compute accuracy
#     accuracy = accuracy_score(ytest, ypred) * 100
#     print("\nAccuracy: %.2f" % accuracy + "%")
# start_time = time.time()
# train_svm()
# print('Execution time: %.2f' % (time.time() - start_time) + ' seconds\n')

In [93]:
def train_svm_with_yolo():
    # list of training and test files
    # call 'datagen' function to get training and testing data & labels
    mlb = MultiLabelBinarizer()
    Xtrain, ytrain = datagen_yolov7(1)
    Xtest, ytest = datagen_yolov7(2)

    # print("ytrain: ", ytrain)
    # convert matrices to numpy array for fast computation
    Xtrain_arr = np.array(Xtrain)
    # convert labels to 1d array to enabel sklearn
    ytrain = [item[0] for item in ytrain]
    # print("x: ", len(Xtrain))
    # print("ytrain: ", ytrain)
    

    Xtest_arr = np.array(Xtest)
    ytest = [np.reshape(y[0], -1) for y in ytest]

    # training phase: SVM , fit model to training data ------------------------------
    clf = svm.SVC(kernel = 'linear')
    clf.fit(Xtrain_arr, ytrain)
    # predict labels for test data
    ypred = clf.predict(Xtest_arr)
    
    # compute accuracy
    accuracy = accuracy_score(ytest, ypred) * 100
    print("\nAccuracy: %.2f" % accuracy + "%")
    
start_time = time.time()
train_svm_with_yolo()
print('Execution time: %.2f' % (time.time() - start_time) + ' seconds\n')


Accuracy: 53.57%
Execution time: 9.53 seconds



#Train with YOLO

---



In [ ]:
# !git clone https://github.com/WongKinYiu/yolov7
# %cd yolov7
# !pip install -r requirements.txt

In [2]:
!pwd
%cd /content

/content
/content


In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone

Cloning into 'yolov5'...
remote: Enumerating objects: 14442, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 14442 (delta 3), reused 2 (delta 0), pack-reused 14430
Receiving objects: 100% (14442/14442), 13.44 MiB | 12.18 MiB/s, done.
Resolving deltas: 100% (9980/9980), done.


/content/yolov5/yolov5


In [ ]:
%pip install -qr requirements.txt  # install

import torch
import utils
display = utils.notebook_init()  # checks

INFO:yolov5:YOLOv5 🚀 v6.2-211-g32a9218 Python-3.7.15 torch-1.8.1+cu101 CPU
YOLOv5 🚀 v6.2-211-g32a9218 Python-3.7.15 torch-1.8.1+cu101 CPU


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 30.2/107.7 GB disk)


In [ ]:
""" Train with YOLOv5"""
rf = Roboflow(api_key="cKIuGvQRsLbBvFgxNztc")
project = rf.workspace("myworkspace-nfnwm").project("projectdb")
dataset = project.version(2).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to ProjectDB-2 in yolov5pytorch:: 100%|██████████| 1144/1144 [00:01<00:00, 1094.04it/s]


In [ ]:
!python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source data/images

detect: weights=['yolov5s.pt'], source=data/images, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-215-g575055c Python-3.7.15 torch-1.8.1+cu101 CPU

100% 14.1M/14.1M [00:00<00:00, 15.6MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
image 1/2 /content/yolov7/yolov5/yolov5/data/images/bus.jpg: 640x480 4 persons, 1 bus, 382.5ms
image 2/2 /content/yolov7/yolov5/yolov5/data/images/zidane.jpg: 384x640 2 persons, 2 ties, 247.6ms
Speed: 4.5ms pre-process, 315.1ms inference, 21.8ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp


In [ ]:
# Download COCO val
torch.hub.download_url_to_file('https://ultralytics.com/assets/coco2017val.zip', 'tmp.zip')  # download (780M - 5000 images)
!unzip -q tmp.zip -d ../datasets && rm tmp.zip  # unzip

  0%|          | 0.00/780M [00:00<?, ?B/s]

In [ ]:
# Validate YOLOv5s on COCO val
!python val.py --weights yolov5s.pt --data /content/yolov7/yolov5/data/coco.yaml --img 640 --half

val: data=/content/yolov7/yolov5/data/coco.yaml, weights=['yolov5s.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=True, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.2-215-g575055c Python-3.7.15 torch-1.8.1+cu101 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
val: Scanning '/content/yolov7/yolov5/datasets/coco/val2017.cache' images and labels... 4952 found, 48 missing, 0 empty, 0 corrupt: 100% 5000/5000 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95:   0% 0/157 [00:00<?, ?it/s]
Traceback (most recent call last):
  File "val.py", line 406, in <module>
    main(opt)
  File "val.py", line 379, in main
    run(**vars(opt))
  File "/usr/local/lib/python3.7/dist-packages/torch/autograd/